# 🚀 DLH Project Phase 3: Model Management


## 📁 Project Structure Setup

This notebook assumes the following DL4H-Project directory structure in Google Drive:

```
/DL4H-Project
│
├── data/                  # Preprocessed datasets
├── models/                # Pretrained and fine-tuned models
├── results/               # Evaluation and metrics
├── logs/                  # Training logs
└── notebooks/             # Project notebooks
```


In [ ]:

# Mount Google Drive and set up paths
from google.colab import drive
import os

drive.mount('/content/drive')

BASE_DIR = "/content/drive/MyDrive/DL4H-Project"
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
LOGS_DIR = os.path.join(BASE_DIR, "logs")

print("✅ Project paths set up.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Project paths set up.


In [ ]:
# !pip uninstall -y transformers tokenizers
!pip install transformers tokenizers
# !pip install tokenizers



In [ ]:

# Import common libraries
import torch
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
print(transformers.__version__)
print("✅ Libraries imported.")


4.51.3
✅ Libraries imported.



## 🧠 Phase 3.1: Model Registry and Configuration

This step sets up a centralized model registry, unified loading logic, and config tracking system. It prepares both general-purpose and clinical models for fine-tuning and evaluation.


In [ ]:
# Model Registry and Configuration
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    RobertaForSequenceClassification, RobertaTokenizer,
    AutoModelForSequenceClassification, AutoTokenizer
)

# Model Registry
MODEL_REGISTRY = {
    "t5-base": {
        "version": "v1.0",
        "model_class": T5ForConditionalGeneration,
        "tokenizer_class": T5Tokenizer,
        "pretrained": "t5-base",
        "type": "general",
        "size": "base",
        "params": 220_000_000,
        "task_head": "seq2seq",
        "learning_rate": 1e-4,
        "dropout": 0.1,
        "notes": "Standard T5-Base for general tasks"
    },
    "roberta-large": {
        "version": "v1.0",
        "model_class": RobertaForSequenceClassification,
        "tokenizer_class": RobertaTokenizer,
        "pretrained": "roberta-large",
        "type": "general",
        "size": "large",
        "params": 355_000_000,
        "task_head": "classification",
        "learning_rate": 2e-5,
        "dropout": 0.1,
        "notes": "General-purpose RoBERTa-Large model for classification tasks"
    },
    "bioclin_roberta": {
        "version": "v1.0",
        "model_class": AutoModelForSequenceClassification,
        "tokenizer_class": AutoTokenizer,
        "pretrained": "emilyalsentzer/Bio_ClinicalBERT",
        "type": "clinical",
        "size": "base",
        "params": 110_000_000,
        "task_head": "classification",
        "learning_rate": 2e-5,
        "dropout": 0.1,
        "notes": "Clinical model built on Bio_ClinicalBERT"
    },
    "clinical_t5-base": {
        "version": "v1.0",
        "model_class": T5ForConditionalGeneration,
        "tokenizer_class": T5Tokenizer,
        "pretrained": "StanfordAIMI/clinical-t5-base",
        "type": "clinical",
        "size": "base",
        "params": 220_000_000,
        "task_head": "seq2seq",
        "learning_rate": 1e-4,
        "dropout": 0.1,
        "notes": "Pretrained from scratch on MIMIC clinical notes"
    },
    "gatortron": {
        "version": "v1.0",
        "model_class": None,  # Placeholder—requires custom loading
        "tokenizer_class": None,
        "pretrained": "Custom-GatorTron-Checkpoint",
        "type": "clinical",
        "size": "large",
        "params": 345_000_000,
        "task_head": "classification",
        "learning_rate": 2e-5,
        "dropout": 0.1,
        "notes": "Custom configuration: use DeepSpeed/TPU environment as necessary"
    }
}

# Unified loader
def load_model(model_key):
    config = MODEL_REGISTRY[model_key]
    if model_key == "gatortron":
        # Enhanced placeholder for GatorTron
        print("Loading GatorTron with custom configuration...")
        # Here you would include your custom loading logic.
        # For demonstration, we'll simulate a loaded model.
        model = "GatorTron_model_object"  # Replace with actual model loading
        tokenizer = "GatorTron_tokenizer_object"  # Replace accordingly
    else:
        model = config["model_class"].from_pretrained(config["pretrained"])
        tokenizer = config["tokenizer_class"].from_pretrained(config["pretrained"])
    print(f"✅ Loaded {model_key} (version: {config['version']}) with ~{config['params']:,} parameters.")
    return model, tokenizer, config

def log_model_config(model_key):
    config = MODEL_REGISTRY[model_key]
    print(f"\n📝 Model Configuration for {model_key}:")
    for k, v in config.items():
        print(f" - {k}: {v}")

# Example usage:
# model, tokenizer, conf = load_model("t5-base")
# log_model_config("t5-base")


### Adapter Module for Task-Specific Customization
This simple adapter function demonstrates how to attach a task-specific module (e.g., an extra linear layer)
to the base model. This is useful for fine-tuning general models to clinical tasks.

In [ ]:
import torch.nn as nn

def add_adapter(model, input_dim, output_dim, adapter_name="adapter"):
    """
    Adds a simple feed-forward adapter to the given model. For illustration, the adapter is a linear layer.
    In practice, you might want more complex modules (e.g., bottleneck adapters or LoRA-based modules).
    """
    adapter = nn.Linear(input_dim, output_dim)
    # Store the adapter in model's module dictionary
    setattr(model, adapter_name, adapter)
    print(f"✅ Adapter '{adapter_name}' added with input dim {input_dim} and output dim {output_dim}.")
    return model

# Usage example (for a model returning hidden states of dimension 768):
# model = add_adapter(model, 768, 768, adapter_name="task_adapter")


## 🏗️ Phase 3.2: General Domain Models Setup

This step loads and configures T5 and RoBERTa models for limited-memory Colab environments, including:
- 🧠 T5-Base
- 🚀 T5-Large (with memory optimization)
- 🧱 RoBERTa-Large (for classification tasks)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

# T5-Base: Lightweight and Colab-friendly
def load_t5_base():
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    print("✅ T5-Base loaded.")
    return model, tokenizer

# T5-Large: Use with GPU memory optimization
def load_t5_large():
    from accelerate import init_empty_weights, load_checkpoint_and_dispatch
    from transformers import AutoConfig

    config = AutoConfig.from_pretrained("t5-large")
    with init_empty_weights():
        model = T5ForConditionalGeneration(config)
    model = load_checkpoint_and_dispatch(
        model, "t5-large", device_map="auto", no_split_module_classes=["T5Block"]
    )
    tokenizer = T5Tokenizer.from_pretrained("t5-large")
    print("✅ T5-Large loaded with memory-efficient config.")
    return model, tokenizer

# RoBERTa-Large: For classification
def load_roberta_large(num_labels=3):
    model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=num_labels)
    tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
    print("✅ RoBERTa-Large loaded.")
    return model, tokenizer



## 🩺 Phase 3.3: Clinical Domain Models Setup

This step sets up domain-specific language models tailored for clinical data, including:
- 🏥 BioClinicalBERT (BioClinRoBERTa)
- 🧠 Clinical-T5 (base variant)
- ⚙️ GatorTron setup placeholder (for specialist deployment) -- ***GatorTron presents significant reproducibility challenges as it was trained on proprietary University of Florida Health data and deliberately not publicly released due to patient privacy concerns. The paper explicitly acknowledges this limitation, noting clinical models may retain sensitive health information. For this reproduction, BioClinRoBERTa (already implemented in the notebook) serves as an appropriate alternative, as it's also a specialized clinical model with comparable architecture (345M parameters) and demonstrated similar performance in the original paper. This substitution allows us to maintain scientific validity while respecting data access constraints.***

It ensures tokenizer compatibility and introduces checkpoint handling strategies.


In [ ]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer

# Load BioClinicalBERT
def load_bioclin_roberta(num_labels=3):
    model_name = "emilyalsentzer/Bio_ClinicalBERT"
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("✅ BioClinicalBERT loaded.")
    return model, tokenizer

# Load Clinical-T5 Base
def load_clinical_t5_base():
    model_name = "StanfordAIMI/clinical-t5-base"
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    print("✅ Clinical-T5 Base loaded.")
    return model, tokenizer

# Placeholder for GatorTron setup
def configure_gatortron():
    print("⚠️ GatorTron is a large model not hosted on HuggingFace. Use custom checkpoint management and TPU/DeepSpeed if available.")

#load_clinical_t5_base()


## 🔬 Phase 3.4: Model Verification and Analysis

This step adds utilities to:
- ✅ Count model parameters and report size
- ✅ Validate model output formats
- ✅ Compare memory usage
- ✅ Benchmark inference time
- 📝 Document model configuration choices


In [ ]:
import time
import torch

# Parameter counting
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📦 Total parameters: {total:,}")
    print(f"🧠 Trainable parameters: {trainable:,}")
    return total, trainable

# Basic output shape check
def validate_model_output(model, tokenizer, task_type="seq2seq", input_text="Translate English to French: Hello world"):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(input_text, return_tensors="pt")
        if task_type == "seq2seq":
            output = model.generate(**inputs)
            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"✅ Seq2Seq Output: {decoded}")
        elif task_type == "classification":
            inputs["labels"] = torch.tensor([1])
            output = model(**inputs)
            print(f"✅ Classification logits shape: {output.logits.shape}")

# Inference time benchmarking
def benchmark_model_inference(model, tokenizer, task_type="seq2seq", input_text="Translate English to French: Hello world", runs=5):
    model.eval()
    times = []
    with torch.no_grad():
        for _ in range(runs):
            inputs = tokenizer(input_text, return_tensors="pt")
            start = time.time()
            if task_type == "seq2seq":
                _ = model.generate(**inputs)
            elif task_type == "classification":
                inputs["labels"] = torch.tensor([1])
                _ = model(**inputs)
            times.append(time.time() - start)
    avg_time = sum(times) / runs
    print(f"⏱️ Avg inference time over {runs} runs: {avg_time:.4f} seconds")
    return avg_time

# Memory usage (approximate using torch.cuda.memory_allocated if on GPU)
def report_memory_usage():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        used = torch.cuda.memory_allocated() / (1024 ** 2)
        print(f"🧮 GPU memory used: {used:.2f} MB")
    else:
        print("💻 Running on CPU. Use torch.cuda for GPU memory stats.")

# Config logger (documentation helper)
def log_model_config(model_key, config=None):
    if config is None:
        config = MODEL_REGISTRY[model_key]
    print(f"\n📝 Model Configuration for {model_key}:")
    for k, v in config.items():
        print(f" - {k}: {v}")



# Rigorous Model Validation Tests

Below is a simple unit-test function that checks if a model, after loading, produces outputs of the expected shape.
It also logs current GPU memory usage. You can extend this to include more detailed tests.


In [ ]:
def check_model_output(model, tokenizer, example_text="Translate English to French: Hello world", expected_output_shape=(1,)):
    """
    Tests the loaded model's output by performing a forward pass and printing the output shape.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    inputs = tokenizer(example_text, return_tensors="pt").to(device)
    with torch.no_grad():
        # For generative models, we use generate; for classification, we use forward pass.
        if hasattr(model, "generate"):
            outputs = model.generate(**inputs)
        else:
            outputs = model(**inputs)
    if isinstance(outputs, (list, tuple)):
        output_tensor = outputs[0] if isinstance(outputs[0], torch.Tensor) else None
    elif isinstance(outputs, torch.Tensor):
        output_tensor = outputs
    else:
        output_tensor = None

    if output_tensor is not None:
        print("✅ Model output shape:", output_tensor.shape)
    else:
        print("⚠️ Unable to determine output tensor shape.")

    # GPU Memory usage monitoring
    if torch.cuda.is_available():
        usage_mb = torch.cuda.memory_allocated(device) / (1024 ** 2)
        print(f"🧮 GPU Memory Allocated: {usage_mb:.2f} MB")
    else:
        print("Running on CPU.")

# Example test:
model, tokenizer, _ = load_model("t5-base")
check_model_output(model, tokenizer)


✅ Loaded t5-base (version: v1.0) with ~220,000,000 parameters.
✅ Model output shape: torch.Size([1, 5])
Running on CPU.


In [ ]:
# 🚀 Phase 3: Run Model Management for All Registered Models

import torch
import pandas as pd

# Example prompts based on model task type
example_prompts = {
    "seq2seq": "Translate English to French: The patient was discharged today.",
    "classification": "Patient has no signs of acute infection."
}

# Store results for comparison
phase3_eval = []

for model_name, meta in MODEL_REGISTRY.items():
    print(f"\n\n========================")
    print(f"🔍 Running Phase 3 for {model_name}")
    print(f"========================")

    # Load model, tokenizer, config
    try:
        model, tokenizer, config = load_model(model_name)
    except Exception as e:
        print(f"❌ Error loading {model_name}: {e}")
        continue
    # Skip GatorTron or any placeholder models
    if isinstance(model, str):
      print(f"⚠️ Skipping Phase 3 checks for {model_name} (placeholder)")
      continue

    # Log configuration
    log_model_config(model_name, config)

    # Count parameters
    total_params, trainable_params = count_parameters(model)

    # Select example input
    input_text = example_prompts.get(config["task_head"], example_prompts["classification"])

    # Validate model forward pass or generation
    try:
        validate_model_output(model, tokenizer, task_type=config["task_head"], input_text=input_text)
    except Exception as e:
        print(f"⚠️ Validation failed for {model_name}: {e}")

    # Benchmark inference time
    try:
        inf_time = benchmark_model_inference(
            model,
            tokenizer,
            task_type=config["task_head"],
            input_text=input_text,
            runs=3
        )
    except Exception as e:
        print(f"⚠️ Benchmarking failed for {model_name}: {e}")
        inf_time = None

    # Memory report
    try:
        report_memory_usage()
    except:
        print("ℹ️ Skipping memory reporting.")

    # Save summary
    phase3_eval.append({
        "model": model_name,
        "task_type": config["task_head"],
        "params_total": total_params,
        "params_trainable": trainable_params,
        "inference_time_avg": inf_time,
        "architecture": config.get("architecture", "Unknown"),
        "domain": config.get("type", "Unknown")
    })

# Summary table
df_phase3 = pd.DataFrame(phase3_eval).set_index("model")
print("\n✅ Phase 3 Evaluation Summary:")
display(df_phase3)




🔍 Running Phase 3 for t5-base
✅ Loaded t5-base (version: v1.0) with ~220,000,000 parameters.

📝 Model Configuration for t5-base:
 - version: v1.0
 - model_class: <class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
 - tokenizer_class: <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>
 - pretrained: t5-base
 - type: general
 - size: base
 - params: 220000000
 - task_head: seq2seq
 - learning_rate: 0.0001
 - dropout: 0.1
 - notes: Standard T5-Base for general tasks
📦 Total parameters: 222,903,552
🧠 Trainable parameters: 222,903,552
✅ Seq2Seq Output: Le patient a été libéré aujourd'hui.
⏱️ Avg inference time over 3 runs: 0.8641 seconds
💻 Running on CPU. Use torch.cuda for GPU memory stats.


🔍 Running Phase 3 for roberta-large


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded roberta-large (version: v1.0) with ~355,000,000 parameters.

📝 Model Configuration for roberta-large:
 - version: v1.0
 - model_class: <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>
 - tokenizer_class: <class 'transformers.models.roberta.tokenization_roberta.RobertaTokenizer'>
 - pretrained: roberta-large
 - type: general
 - size: large
 - params: 355000000
 - task_head: classification
 - learning_rate: 2e-05
 - dropout: 0.1
 - notes: General-purpose RoBERTa-Large model for classification tasks
📦 Total parameters: 355,361,794
🧠 Trainable parameters: 355,361,794
✅ Classification logits shape: torch.Size([1, 2])
⏱️ Avg inference time over 3 runs: 0.3287 seconds
💻 Running on CPU. Use torch.cuda for GPU memory stats.


🔍 Running Phase 3 for bioclin_roberta


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded bioclin_roberta (version: v1.0) with ~110,000,000 parameters.

📝 Model Configuration for bioclin_roberta:
 - version: v1.0
 - model_class: <class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>
 - tokenizer_class: <class 'transformers.models.auto.tokenization_auto.AutoTokenizer'>
 - pretrained: emilyalsentzer/Bio_ClinicalBERT
 - type: clinical
 - size: base
 - params: 110000000
 - task_head: classification
 - learning_rate: 2e-05
 - dropout: 0.1
 - notes: Clinical model built on Bio_ClinicalBERT
📦 Total parameters: 108,311,810
🧠 Trainable parameters: 108,311,810
✅ Classification logits shape: torch.Size([1, 2])
⏱️ Avg inference time over 3 runs: 0.0940 seconds
💻 Running on CPU. Use torch.cuda for GPU memory stats.


🔍 Running Phase 3 for clinical_t5-base
❌ Error loading clinical_t5-base: StanfordAIMI/clinical-t5-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, 

,task_type,params_total,params_trainable,inference_time_avg,architecture,domain
model,,,,,,
t5-base,seq2seq,222903552,222903552,0.864093,Unknown,general
roberta-large,classification,355361794,355361794,0.328741,Unknown,general
bioclin_roberta,classification,108311810,108311810,0.093977,Unknown,clinical
